### The aim of the project is to build an LSTM model to classify fake news based on title of the data

### Import necessary libraries and modules

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


### Data mining

The dataset is obtained from Kaggle.
https://www.kaggle.com/competitions/fake-news/data?select=submit.csv

Reading the data and performing basic EDA

In [2]:
df = pd.read_csv(r"D:\dataset\fake news\train.csv")

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.shape

(20800, 5)

In [5]:
df = df.dropna()

In [6]:
df.shape

(18285, 5)

#### We need to separate the data into dependent and independent category, which essentially means dropping the label column for dependent data

In [7]:
X =df.drop(['label'], axis=1)

In [8]:
y = df['label']

In [9]:
y.shape

(18285,)

#### Importing the modules required to build LSTM model

In [10]:
import tensorflow as tf

In [11]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [12]:
voc_size = 5000

In [13]:
messages = X.copy()

In [14]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [15]:
messages.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [16]:
messages.reset_index(inplace=True)

#### Import libraries for sentiment analysis

In [17]:
import nltk
import re
from nltk.corpus import stopwords

#### We need to clean the text data by lowering the words to avoid repetition of words, and by removing stopwords as they play no part in sentiment analysis

In [18]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

corpus = []
for i in range(0, len(messages)):
   
    review = re.sub('[^a-zA-Z]',' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [19]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

#### Now we need to convert each word to a vector as form of one-hot representation.

In [20]:
onehot_repr =[one_hot(words,voc_size) for words in corpus]
onehot_repr

[[2185, 885, 1828, 1196, 4349, 1219, 2380, 1788, 2358, 4538],
 [825, 1478, 1113, 3695, 113, 918, 2535],
 [4213, 256, 876, 1617],
 [3180, 743, 2993, 2297, 2464, 3103],
 [2840, 113, 386, 792, 1304, 4412, 113, 3342, 4051, 4472],
 [4612,
  197,
  2406,
  3488,
  1066,
  612,
  444,
  3690,
  4416,
  2722,
  3552,
  1453,
  2503,
  1976,
  2535],
 [4035, 1714, 4821, 235, 3065, 133, 3095, 3075, 4407, 1093, 4825],
 [2471, 741, 2820, 4218, 4257, 3814, 612, 4102, 4407, 1093, 4825],
 [1064, 1110, 804, 1145, 582, 341, 2372, 2013, 612, 3624],
 [122, 4559, 2523, 3170, 2054, 2398, 1384, 2668],
 [2422, 272, 3025, 2183, 489, 3180, 778, 1946, 1726, 9, 371],
 [2297, 3284, 4349, 341, 612, 4257],
 [2010, 2028, 2819, 2322, 1046, 1015, 2292, 89, 3170],
 [1941, 896, 966, 2036, 3081, 983, 4660, 4407, 1093, 4825],
 [4601, 445, 2330, 1559, 4498, 4407, 1093, 4825],
 [2598, 2471, 259, 1013, 2166, 2201, 2417, 1419, 836, 4449],
 [2753, 1665, 1478],
 [341, 3365, 3600, 1048, 612, 1676, 4791, 2535],
 [96, 3970, 1113, 

#### We can see that length of each sentence is different. Therefore, we need to use padding to make it uniform.

In [21]:
sent_length = 20
embedded_docs = pad_sequences(onehot_repr, padding='pre', maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 1788 2358 4538]
 [   0    0    0 ...  113  918 2535]
 [   0    0    0 ...  256  876 1617]
 ...
 [   0    0    0 ... 4407 1093 4825]
 [   0    0    0 ... 2541 1086 4639]
 [   0    0    0 ... 1809 3786 4752]]


### Compiling the model

In [22]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [23]:
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [24]:
X_final.shape, y_final.shape

((18285, 20), (18285,))

### Sampling the data and splitting it into train and test sets

In [26]:
from sklearn.model_selection import train_test_split

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [28]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 12s 39ms/step - loss: 0.3293 - accuracy: 0.8500 - val_loss: 0.1992 - val_accuracy: 0.9186
Epoch 2/10
192/192 [==============================] - 7s 37ms/step - loss: 0.1387 - accuracy: 0.9450 - val_loss: 0.1965 - val_accuracy: 0.9215
Epoch 3/10
192/192 [==============================] - 7s 35ms/step - loss: 0.0979 - accuracy: 0.9652 - val_loss: 0.2170 - val_accuracy: 0.9216
Epoch 4/10
192/192 [==============================] - 6s 34ms/step - loss: 0.0707 - accuracy: 0.9762 - val_loss: 0.2580 - val_accuracy: 0.9155
Epoch 5/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0448 - accuracy: 0.9856 - val_loss: 0.3061 - val_accuracy: 0.9183
Epoch 6/10
192/192 [==============================] - 7s 35ms/step - loss: 0.0285 - accuracy: 0.9909 - val_loss: 0.3071 - val_accuracy: 0.9082
Epoch 7/10
192/192 [==============================] - 8s 42ms/step - loss: 0.0193 - accuracy: 0.9938 - val_loss: 0.4043 - val_accuracy: 0.914

epochs and batch size are hyper parameters, which need to be varied according to the performance of the model

In [29]:
y_pred = model.predict(X_test)


189/189 [==============================] - 2s 8ms/step


In [30]:
y_pred = np.where(y_pred > 0.5, 1,0) 

#### Whenever the probability is greater than 0.5 it will be classified as fake news.

#### Building a confusion matrix to evaluate performance of the model.

In [31]:
from sklearn.metrics import confusion_matrix


In [32]:
confusion_matrix(y_test, y_pred)

array([[3133,  286],
       [ 255, 2361]], dtype=int64)

In [33]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9103562551781276

It can be noticed that the accuracy score obtained is high 

In [34]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.92      0.92      3419
           1       0.89      0.90      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035



The precision score, recall score and f1-score are showing a good sign and are thus suitable for the model building.